# Fantasy Hockey on D-Wave

In this notebook, we will go through how to perform a Markowitz style portfolio optimization using a D-Wave Quantum Annealer, as opposed to traditional Linear/Quadradic programming solvers. To begin, we first need to define our problem in terms of how it would work as a traditional constrained optimization problem, then how to re-write it as an "unconstrained" problem that can be translated as a Binary Quadratic Model (BQM) to be run on D-Wave quantum processors. 

## The Model

If we were taking the classical approach using constrained optimizers, our problem would formally appear as follows

$$
\begin{aligned}
\text{maximize    } \;\;& \mathbf{r x} - \gamma \mathbf{x}^T \mathbf{Qx} \\
\text{subject to  } \;\;& \sum_{x_i \in G} x_i = N_G \\
& \sum_{x \in T} C_i x_i = V \\
& \sum_{x_i \in F} x_i= N_F \\
& \sum_{x_i \in D} x_i = N_D \\
& \mathbf{x} \in \mathbb{B}
\end{aligned}
$$

Where $\mathbf{r}$ is a returns vector, here defined as the average returns, $\mathbf{x}$ is a binary vector of players, and $\mathbf{Q}$ is a covariance matrix of returns. The constraint equations above imply that we _must_ choose some number of players in each position, in this case the indexes $F,D$ and $G$ mean forwards, defence and goalies respectively. As well, we have a point values constraint as represented by $ \sum_{x \in T}C_i x_i = V $ where $T$ is the team we have assembled, $C_i$ is the cost of the $i^{th}$ player, and $V$ is the amount of points we can "spend" to draft our fantasy team. 

To make the translation to a problem that we can run on D-Wave, we first rewrite the equation above outside of matrix notation and write the sums explicitly

$$
\begin{aligned}
\text{maximize    } \;\;& \sum_{i=1}^N r_i x_i  - \gamma \sum_{i=1}^N \sum_{j=1}^N q_{ij} x_i x_j \\
\text{subject to  } \;\;& \sum_{x_i \in G} x_i = N_G \\
& \sum_{x \in T} C_i x_i = V \\
& \sum_{x_i \in F} x_i= N_F \\
& \sum_{x_i \in D} x_i = N_D \\
& \mathbf{x} \in \mathbb{B}
\end{aligned}
$$

Where $N$ is the number of players. At this point, we have to get sneaky. As we cannot specify our constraints in the form we have written here, we have to re-write our above equation in an unconstrained form. Explicitly, we don't want to change our equation, that would change our objective function, and therefore, our optimization. In this case we have two options

1. Multiply by 1
2. Add zero

However, there will be something special about these ones and zeros in the sense that they will only be zero when our constraints our satisfied. Explicitly, we will subract the following from our objective function

$$ 
 \lambda_1 \sum_\alpha \left( N_\alpha - \sum_i C_{\alpha, i} x_i \right)^2 + \lambda_2\left(V - \sum_i v_i x _i \right) ^2
$$

where $\lambda_1$ and $\lambda_2$ are Lagrange multipliers, $\alpha$ is a subscript representing $F,G$ and $D$ - meaning that this is now our positional constraint. In this case the term $C_{\alpha, i}$ represents the "position cost" of a particular player. That is to say if a player is a defenceman, than the "costs" associated with this player are as follows

| Constant   | Value for a Defence man  |
|------------|--------------------------|
| $C_{F,i}$  | 0                        |
| $C_{G, i}$ | 0                        |
| $C_{D, i}$ | 1                        |

Where from the table above, we see that each constant is 1 if a player is the appropriate $\alpha$ subscript, and 0 else. This allows us to re-write our positional constraints in an "unconstrained" manner that will enforce that we select a valid team. The second term in the new piece represents the point value constraint, in that the cost of each player we have must all add up to our buy limit of $V$.  

The two terms above are squared in order to enforce that the minimum of these two terms is strictly zero, and when our constraints are satisfied each of these terms should be zero. 

Written in full our new optimization problem is

$$
\text{maximize} \; \;\sum_{i=1}^N r_i x_i  - \gamma \sum_{i=1}^N \sum_{j=1}^N q_{ij} x_i x_j - \lambda_1 \sum_\alpha \left( N_\alpha - \sum_i C_{\alpha, i} x_i \right)^2 - \lambda_2 \left(V - \sum_i v_i x _i \right) ^2
$$

Of course, we have to write the above in terms of a Binary Quadratic Model of the form

$$ \sum_i a_i x_i - \sum_i \sum_j b_{i,j} x_i x_j $$

In order to do this, we have to expand the squared terms in our problem which is done below

$$
\begin{aligned}
\sum_\alpha \left( N_\alpha - \sum_i C_{\alpha, i} x_i \right)^2 & = \sum_\alpha \left( N_\alpha - \sum_i C_{\alpha, i} x_i \right)\left( N_\alpha - \sum_i C_{\alpha, i} x_i \right) \\
& = \sum_\alpha \left(N_\alpha ^2 + \sum_i \sum_j C_{\alpha, i} C_{\alpha, j} x_i x_j\right) - 2 \sum_\alpha N_\alpha \sum_i C_{\alpha, i } x_i 
\end{aligned}
$$

And similarily for the player-value constraint

$$
\begin{aligned}
\left(V - \sum_i v_i x _i \right) ^2 & = \left(V - \sum_i v_i x_i \right) \left(V - \sum_i v_i x _i \right) \\
& = V^2 + \sum_i \sum_j v_i v_j x_i x_j - 2 V \sum_i v_i x_i 
\end{aligned}
$$


Where, before writing the final expression, we note that the constants $N_\alpha^2$ and $V^2$ represent a constant offset and won't change the true results of the optimization, we will drop them from the problem. 

Our final objective function is as follows 

$$
\begin{aligned} 
\text{maximize } & \; \sum_i \left(r_i + 2 \lambda_1 \sum_\alpha N_\alpha C_{\alpha,i} + 2 \lambda_2 V v_i\right) x_i  \\
& -\sum_i \sum_j \left( \gamma q_{ij} + \lambda_1 \sum_\alpha C_{\alpha, i} C_{\alpha, j} + \lambda_2 v_i v_j \right) x_i x_j \\
& x \in \mathbb{B}
\end{aligned}
$$

The file `../scripts/src.py` contains the functions `linearTerms` and `quadraticTerms`, which we will use to calculate the constants in front of both the linear and quadratic terms respectively. The rest of this notebook demonstrates how to set up the data, and solve the problem using `dimod`, the python package which contains the required functions to complete a problem on a `D-Wave` system.

I should note that I wouldn't recommend using this to _actually_ try and win at fantasy sports. An optimization like this would absolutely perform above average in a real set up - but the way I have chosen to set up the data neglects a lot of important covariances between players on opposing teams, and more importantly, some terms which would reward choosing players on the same line would be important. As This is set up for a very simple scoring system of 1 point per goal/assist for players, and 2 points for a goalie win, and 1 for a shut out, if we put similar lines together when possible we could gain more points by "double diping" the score/assist points in our fantasy line up. 

## A note on Lagrange Multipliers

In the following analysis we will be setting $\lambda_1 = 1$ and $\lambda_2 = 0.1$. These values are artificial and could be adjusted, however, what they do represent is approximately "how much we care" about each constraint. In thise case, we care a lot of we have the right number of players - if we choose poorly we will be very bad at fantasy hockey. However, if we choose players such that their points value is _less_ than our budget - that's actually okay - so this is a statement of attempting to punish those violations less severely when it works out well to use "cheaper" players. 

# Data Set Up

Here we are not focusing too much on any data wrangling tasks - instead we're focusing on the solutions as described above and using D-Wave solvers to find a solution. Our first step is to import our data and required packages

In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '../')
import scripts.src as src
import dimod

Here we are importing some data pre-wrangled (mostly). `player_data` is various statistics of each player, `all_points` is a csv organized like 

```
PlayerName1, [list of all points in season]
...
...
```

Which allows us to see the variance and statistics of each player. Finally `pvals` is the points value of each player according to the current (as of August 2020) SportsNet Fantasy Playoff draft.

In [2]:
player_data = pd.read_csv('../data/player_data.csv')
del player_data['Unnamed: 0']
all_points = pd.read_csv('../data/all_points.csv')
del all_points['Unnamed: 0']
pvals = pd.read_csv('../data/pvals.csv')
pvals = pvals.set_index('name')

Here we are creating our "position costs" data set which will allow us to effectively model player constraints as an unconstrained BQM. 

In [3]:
players = player_data[['name','position']].drop_duplicates(subset='name')

players['costs'] = players.apply(src.costFiller, axis=1)
players[['CD','CF','CG']] = pd.DataFrame(players.costs.tolist(), 
                                          index= players.index)
del players['costs']
del players['position']
players = players.set_index('name')
players.head()

,CD,CF,CG
name,,,
Adam Boqvist,1,0,0
Adam Gaudette,0,1,0
Adam Pelech,1,0,0
Adam Werner,0,0,1
Adin Hill,0,0,1
...,...,...,...
Yanni Gourde,0,1,0
Zach Bogosian,1,0,0
Zach Werenski,1,0,0


## Setting Up the BQM

The functions below define our BQM. Our linear ($x_i$) terms are created as a dictionary from `src.linearTerms`, and our quadratic ($x_i x_j$) terms are created as a dictonary from `src.quadraticTerms`. 

In [39]:

num_each = {'CD':4, 'CF':6, 'CG': 2}
# Generating our returns and covariance terms for Markowitz Style Optimizations
rets = all_points.mean()#.sample(100)
cov  = all_points[list(rets.keys())].cov()
    
lins = src.linearTerms(rets, players, num_each,pvals)
quads = src.quadraticTerms(cov, players,pvals)


As a sanity check (and to make sure we don't have any mistakes before we waste our precious QPU time) we first check with the `SimmulatedAnnealingSampler` to see if we can find a solution. If you're testing this, this takes a while with the full set of players chosen (N $\approx$ 300), and I recommend uncommenting the `#sample(100)` to make things a little faster. It should also be noted that the simulated annealing sampler probably isn't going to converge to a good solution in reasonable time. Certainly, you could adjust `num_sweeps` to be larger, but I think you'd have to be pretty patient regardless. That said, if you're looking for an excuse to take a break, I absolutely recommend setting `num_sweeps` to something much higher. 

In [40]:
%%time
# Appears to scale > N^2, where N is the number of players
import dimod 
bqm = dimod.BQM(lins, 
                quads,
                0,
                dimod.BINARY)

test = dimod.SimulatedAnnealingSampler().sample(bqm, num_sweeps = 500)

CPU times: user 6min 25s, sys: 820 ms, total: 6min 26s
Wall time: 6min 27s


Let's take a look at our team: 

In [41]:
choices = list(test.first.sample.items())
filt = []
for name in choices:
    if name[1] == 1: 
        filt.append(name[0])
x = player_data[player_data.name.isin(filt)][['name', 'position', 'PV']].drop_duplicates(subset = 'name')
x

,name,position,PV
1022,Alex Killorn,L,2
1481,Alex Ovechkin,L,3
1634,Alex Pietrangelo,D,2
3870,Andy Greene,D,1
4033,Anthony Cirelli,C,2
5442,Bo Horvat,C,2
7086,Brian Elliott,G,3
7392,Brock Nelson,C,2
13411,Dougie Hamilton,D,2
17220,Jake Allen,G,3


In [42]:
len(filt), x.PV.sum()

(12, 27)

# Using the QPU
Now that things look like they're working, let's check the speed up (and differences in solution) by using the cloud-based QPU solvers from dwave. First I note that you first will have to sign up for a [LEAP](https://www.dwavesys.com/take-leap) account, and set up your API access with instructions at [this link](https://docs.ocean.dwavesys.com/en/latest/overview/sapi.html)

## TODO
This is all using default parameters in LeapHybridSampler - certainly we would do even better with a little bit of tuning of parameters.

In [9]:
# I don't think this is required but now I'm scared to remove it. 
from dwave.cloud import Client
client = Client.from_config(token='YOUR API KEY HERE') 
client.get_solvers()     


[StructuredSolver(id='DW_2000Q_6'), UnstructuredSolver(id='hybrid_v1')]

In [51]:
%%time 
from dwave.system import LeapHybridSampler

sampler = LeapHybridSampler()

quantum_test = sampler.sample(bqm)


CPU times: user 314 ms, sys: 93.1 ms, total: 407 ms
Wall time: 4.84 s


In [52]:
Q_choices = list(quantum_test.first.sample.items())
filt2 = []
for name in Q_choices:
    if name[1] == 1: 
        filt2.append(name[0])
x2 = player_data[player_data.name.isin(filt2)][['name', 'position', 'PV']].drop_duplicates(subset = 'name')
x2

,name,position,PV
1175,Alex Lyon,G,2
1481,Alex Ovechkin,L,3
5442,Bo Horvat,C,2
6686,Brayden Point,C,3
6839,Brayden Schenn,C,2
15612,Gabriel Landeskog,L,2
17747,Jakub Voracek,R,2
19221,John Carlson,D,3
23897,Mark Giordano,D,2
36805,Thomas Greiss,G,3


In [54]:
len(filt), x.PV.sum()

(12, 27)

Where we see that the teams are a little different between the two cases. However, what we should note is that the QPU was 

In [53]:
(6*60 +  25) / 4.81

80.04158004158005

Time faster than the other example. It should also be noted that a lot of the time associated with the QPU time is server connection time etc. 

That said, this may not be a fair comparison as you would likely treat this problem using a more sophisticated classical quadratic programming solver (such as one from [cvxpy](https://www.cvxpy.org/) ).